# SOMocluSummarizer+Quality Control demo

Author: Ziang Yan <br>
Last successfully run: Nov 22, 2024<br>

This notebook creats an end-to-end example for the SOM summarizer PLUS quality controld defined in https://arxiv.org/pdf/2007.15635. Including:

1) create photometric realizations for a training and spectroscopic sample;
2) measuring BPZ for the training and spectroscopic samples;
3) make the same tomographic cut on the training and spec samples;
4) informing a `rail_som` model with the training sample and summarizing it with the spec sample;
5) performing two quality control (arXiv: 1909.09632);
6) summarizing the goodness of redshift calibration and compare between QCs;


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pickle
import rail
import os
import qp
from rail.core.utils import RAILDIR

import tables_io
from rail.core.data import TableHandle, ModelHandle
from rail.core.stage import RailStage
from rail.estimation.algos.somoclu_som import SOMocluInformer, SOMocluSummarizer
from rail.estimation.algos.somoclu_som import get_bmus, plot_som

Next, let's set up the Data Store, so that our RAIL module will know where to fetch data:

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

First, let's grab some data files.  For the SOM, we will want to train on a fairly large, representative set that encompasses all of our expected data.  We'll grab a larger data file than we typically use in our demos to ensure that we construct a meaningful SOM.

## Run this command on the command line to get the larger data file to train the SOM:
`curl -O https://portal.nersc.gov/cfs/lsst/schmidt9/healpix_10326_bright_data.hdf5`

and then move the resulting file to this directory, i.e. RAIL/examples/estimation.  This data consists of ~150,000 galaxies from a single healpix pixel of the comsoDC2 truth catalog with mock 10-year magnitude errors added.  It is cut at a relatively bright i<23.5 magnitudes in order to concentrate on galaxies with particularly high S/N rates.

# First read the target and spec catalogue from a pre-trained pzflow stage.

In [ ]:
training_file = "./healpix_10326_bright_data.hdf5"

if not os.path.exists(training_file):
  os.system('curl -O https://portal.nersc.gov/cfs/lsst/PZ/healpix_10326_bright_data.hdf5')


In [ ]:
training_data = DS.read_file("training_data", TableHandle, training_file)

In [ ]:
pmask = (training_data.data['photometry']['mag_i_lsst'] <23.5)
trim_test = {}
for key in training_data.data['photometry'].keys():
    trim_test[key] = training_data.data['photometry'][key][pmask]
trim_dict = dict(photometry=trim_test)
target_data_all = DS.add_data("target_data_raw", trim_dict, TableHandle)

In [ ]:
from rail.utils.path_utils import find_rail_file

specfile = find_rail_file("examples_data/testdata/test_dc2_validation_9816.hdf5")
ref_data_raw = tables_io.read(specfile)['photometry']
smask = (ref_data_raw['mag_i_lsst'] <23.5)
trim_spec = {}
for key in ref_data_raw.keys():
    trim_spec[key] = ref_data_raw[key][smask]
trim_dict = dict(photometry=trim_spec)
ref_data_all = DS.add_data("ref_data_raw", trim_dict, TableHandle)

# Now measure the photometric redshifts using the `bpz_lite`

In [ ]:
bands = ["u", "g", "r", "i", "z", "y"]
lsst_bands = []
lsst_errs = []
lsst_filts = []
for band in bands:
    lsst_bands.append(f"mag_{band}_lsst")
    lsst_errs.append(f"mag_err_{band}_lsst")
    lsst_filts.append(f"DC2LSST_{band}")
print(lsst_bands)
print(lsst_filts)

In [ ]:
from rail.core.utils import RAILDIR
import os
from rail.core.utils import RAILDIR
from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator
from rail.core.data import ModelHandle
custom_data_path = RAILDIR + '/rail/examples_data/estimation_data/data'

hdfnfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/CWW_HDFN_prior.pkl")
sedfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/SED/COSMOS_seds.list")

with open(hdfnfile, "rb") as f:
    hdfnmodel = pickle.load(f)

custom_dict_phot = dict(hdf5_groupname="photometry",
                   output="bpz_results_phot_qc.hdf5", 
                   bands=lsst_bands, 
                   err_bands=lsst_errs,
                   filter_list=lsst_filts,
                   prior_band='mag_i_lsst',spectra_file=sedfile,
                   data_path=custom_data_path,
                   no_prior=False)

custom_dict_spec = dict(hdf5_groupname="photometry",
                   output="bpz_results_spec_qc.hdf5", 
                    bands=lsst_bands, 
                   err_bands=lsst_errs,
                   filter_list=lsst_filts,
                   prior_band='mag_i_lsst',spectra_file=sedfile,
                   data_path=custom_data_path,
                   no_prior=False)

cosmospriorfile = os.path.join(RAILDIR, "rail/examples_data/estimation_data/data/COSMOS31_HDFN_prior.pkl")
cosmosprior = DS.read_file("cosmos_prior", ModelHandle, cosmospriorfile)

phot_run = BPZliteEstimator.make_stage(name="rerun_bpz_phot", model=cosmosprior, **custom_dict_phot)
spec_run = BPZliteEstimator.make_stage(name="rerun_bpz_spec", model=cosmosprior, **custom_dict_spec)

In [ ]:
from collections import OrderedDict
phot_run.estimate(target_data_all)

In [ ]:
spec_run.estimate(ref_data_all)

In [ ]:
phot_bpz_file = 'bpz_results_phot_qc.hdf5'
bpz_phot_all = tables_io.read(phot_bpz_file)['ancil']['zmode']

spec_bpz_file = 'bpz_results_spec_qc.hdf5'
bpz_spec_all = tables_io.read(spec_bpz_file)['ancil']['zmode']

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(target_data_all.data['photometry']['redshift'], bpz_phot_all, s=0.3)
plt.plot(target_data_all.data['photometry']['redshift'],target_data_all.data['photometry']['redshift'], color='C1')
plt.title('Test data')
plt.xlabel(r'$Z_{\mathrm{spec}}$', fontsize=15)
plt.ylabel(r'$Z_{\mathrm{phot}}$', fontsize=15)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(ref_data_all.data['photometry']['redshift'], bpz_spec_all, s=0.3)
plt.plot(ref_data_all.data['photometry']['redshift'],ref_data_all.data['photometry']['redshift'], color='C1')
plt.title('Spec data')
plt.xlabel(r'$Z_{\mathrm{spec}}$', fontsize=15)
plt.ylabel(r'$Z_{\mathrm{phot}}$', fontsize=15)

## cut the data to make a tomographic bin

In [ ]:
bin_low = 0.2
bin_high = 0.5

In [ ]:
trim_data_test = {}

mask_phot = ((bpz_phot_all > bin_low) & (bpz_phot_all < bin_high))
mask_phot &= (target_data_all.data['photometry']['redshift'] > 0)

bpz_phot = bpz_phot_all[mask_phot]

for key in target_data_all.data['photometry'].keys():
    trim_data_test[key] = target_data_all.data['photometry'][key][mask_phot]
trimdict_test = dict(photometry=trim_data_test)
target_data = DS.add_data("testing_data", trimdict_test, TableHandle)

In [ ]:
trim_data_spec = {}

mask_spec = ((bpz_spec_all > bin_low) & (bpz_spec_all<bin_high))
mask_spec &= (ref_data_all.data['photometry']['redshift'] > 0)

bpz_spec = bpz_spec_all[mask_spec]

for key in target_data_all.data['photometry'].keys():
    trim_data_spec[key] = ref_data_all.data['photometry'][key][mask_spec]
trimdict_spec = dict(photometry=trim_data_spec)
ref_data = DS.add_data("ref_data", trimdict_spec, TableHandle)

In [ ]:
plt.title('Redshift distributions')
plt.xlabel(r'$Z_{\mathrm{spec}}$')
plt.ylabel('dN/dz')

plt.hist(target_data.data['photometry']['redshift'], bins=50, density=True, histtype='step', label='Target')
plt.hist(ref_data.data['photometry']['redshift'], bins=50, density=True, histtype='step', label='Reference')
plt.axvline(bin_low, color='k', linestyle='--')
plt.axvline(bin_high, color='k', linestyle='--')
plt.legend()

# Now let's train a SOM with the color from the target set

We need to define all of our necessary initialization params, which includes the following:
- `name` (str): the name of our estimator, as utilized by ceci
- `model` (str): the name for the model file containing the SOM and associated parameters that will be written by this stage
- `hdf5_groupname` (str): name of the hdf5 group (if any) where the photometric data resides in the training file
- `n_rows` (int): the number of dimensions in the y-direction for our 2D SOM
- `n_columns` (int): the number of dimensions in the x-direction for our 2D SOM
- `gridtype` (str): the parameter that specifies the grid form of the nodes. Options: `rectangular`(default) and `hexagonal`.
- `initialization` (str): the parameter specifying the method of initializing the SOM. Options: `pca`: principal componant analysis (default); `random`: randomly initialize the SOM.
- `maptype` (str): the parameter specifying the map topology. Options: `planar`(default) and `toroid`.
- `n_epochs` (int): the number of iteration steps during SOM training.  SOMs can take a while to converge, so we will use a fairly large number of 500,000 iterations.
- `std_coeff` (float): the "radius" of how far to spread changes in the SOM 
- `som_learning_rate` (float): a number between 0 and 1 that controls how quickly the weighting function decreases.  SOM's are not guaranteed to converge mathematically, and so this parameter tunes how the response drops per iteration.  A typical values we might use might be between 0.5 and 0.75.
- `column_usage` (str):  this value determines what values will be used to construct the SOM, valid choices are `colors`, `magandcolors`, and `columns`.  If set to `colors`, the code will take adjacent columns as specified in `usecols` to construct colors and use those as SOM inputs.  If set to `magandcolors` it will use the single column specfied by `ref_column_name` and the aforementioned colors to construct the SOM.  If set to `columns` then it will simply take each of the columns in `usecols` with no modification.  So, if a user wants to use K magnitudes and L colors, they can precompute the colors and specify all names in `usecols`.  NOTE: accompanying `usecols` you must have a `nondetect_val` dictionary that lists the replacement values for any non-detection-valued entries for each column, see the code for an example dictionary.  WE will set `column_usage` to colors and use only colors in this example notebook.

In [ ]:
dim = 101
grid_type = 'hexagonal'


inform_dict = dict(model='output_SOMoclu_model.pkl', 
                   hdf5_groupname='photometry',
                   n_rows=dim, n_columns=dim, 
                   gridtype = grid_type,
                   maptype = 'toroid',
                   n_epochs=30,
                   std_coeff=12.0, som_learning_rate=0.75,
                   column_usage='colors')

inform_som = SOMocluInformer.make_stage(name='inform_som', **inform_dict)

In [ ]:
%%time
inform_som.inform(target_data)

In [ ]:
def get_cont_hist(data, bins):
    hist, bin_edge = np.histogram(data, bins=bins, density=True)
    return hist, (bin_edge[1:]+bin_edge[:-1])/2

# Summarize the SOM with target data and spectroscopic reference

In [ ]:
from scipy.stats import norm

n_clusters = 1000

summ_dict = dict(model="output_SOMoclu_model.pkl", hdf5_groupname='photometry',
                 spec_groupname='photometry', nzbins=101, nsamples=25,
                 output='SOM_ensemble.hdf5', single_NZ='fiducial_SOMoclu_NZ.hdf5',
                 n_clusters=n_clusters,
                 uncovered_cluster_file ='all_uncovered_cells.hdf5',
                 objid_name='id',
                 cellid_output='output_cellIDs.hdf5')

som_summarizer = SOMocluSummarizer.make_stage(name='SOMoclu_summarizer', aliases=dict(model="model_somoclu__"), **summ_dict)
som_summarizer.summarize(target_data, ref_data)


We can calculate the best SOM cell using the get_bmus() function defined in somocluSOM.py, which will return the 2D SOM coordinates for each galaxy. Then we group the SOM cells into hierarchical clusters and calculate the occupation and mean redshift in each cluster. 

To do this, we first get the colors for the reference and target sample.

In [ ]:
bands = ['u','g','r','i','z','y']
bandnames = [f"mag_{band}_lsst" for band in bands]

ngal_ref = len(ref_data.data['photometry']['mag_i_lsst'])
ngal_target = len(target_data.data['photometry']['mag_i_lsst'])

ref_colors = np.zeros([5, ngal_ref])
target_colors = np.zeros([5, ngal_target])
for i in range(5):

    ref_colors[i] = ref_data.data['photometry'][bandnames[i]] - ref_data.data['photometry'][bandnames[i+1]]
    target_colors[i] = target_data.data['photometry'][bandnames[i]] - target_data.data['photometry'][bandnames[i+1]]

In [ ]:
with open("output_SOMoclu_model.pkl", "rb") as f:
    model = pickle.load(f)

Now we call the `get_bmus` function to get the best matching units for the reference and target data.

In [ ]:
SOM = model['som']
ref_bmu_coordinates = get_bmus(SOM, ref_colors.T, 1000).T
target_bmu_coordinates = get_bmus(SOM, target_colors.T, 1000).T

Find the cluster index for each galaxy in the reference and target data

In [ ]:
import sklearn.cluster as sc

algorithm = sc.AgglomerativeClustering(n_clusters=n_clusters, linkage='complete')
SOM.cluster(algorithm)
som_cluster_inds = SOM.clusters.reshape(-1)

ref_pixel_coords = np.ravel_multi_index(ref_bmu_coordinates, (dim, dim))
ref_som_clusterind = som_cluster_inds[ref_pixel_coords]

target_pixel_coords = np.ravel_multi_index(target_bmu_coordinates, (dim, dim))
target_som_clusterind = som_cluster_inds[target_pixel_coords]

The next cell defines some functions to plot cluster boundaries on a SOM grid. This function will be added to rail_som in a subsequent update. 

In [ ]:
def plot_cluster_boundaries(ax, SOM, n_clusters, cluster_inds=None, topology='hexagonal'):
    dim = SOM.codebook.shape[0]
    som_cluster_ind = SOM.clusters.reshape(-1)
    som_centers = find_cell_centers(dim, topology=topology)
    som_centers_l = np.array([som_centers.T[0]-dim*1, som_centers.T[1]]).T
    som_centers_r = np.array([som_centers.T[0]+dim*1, som_centers.T[1]]).T
    som_centers_u = np.array([som_centers.T[0], som_centers.T[1]+dim*np.sqrt(3)/2]).T
    som_centers_b = np.array([som_centers.T[0], som_centers.T[1]-dim*np.sqrt(3)/2]).T
    if cluster_inds is None:
        cluster_inds = np.arange(n_clusters)
    for i in (cluster_inds):
        centers = (np.vstack([som_centers[np.where(som_cluster_ind==i)[0]],
                         som_centers_l[np.where(som_cluster_ind==i)[0]], som_centers_u[np.where(som_cluster_ind==i)[0]], 
                         som_centers_r[np.where(som_cluster_ind==i)[0]], som_centers_b[np.where(som_cluster_ind==i)[0]]]))
        linep = get_manycells_boundary(centers, topology='hexagonal')
        for points in linep:
            if points.T[0].min() < som_centers.T[0].min()-1 or points.T[0].max() > som_centers.T[0].max()+1 or points.T[1].min() < som_centers.T[1].min()-np.sqrt(3)/2 or points.T[1].max() > som_centers.T[1].max()+np.sqrt(3)/2:
            #plt.plot(points.T[0], points.T[1], color='blue')
                continue
            ax.plot(points.T[0], points.T[1], color='k', lw=0.2)
    return

def find_cell_centers(dim, topology='rectangular'):
    if topology == 'rectangular':
        x = np.arange(dim) + 0.5
        y = np.arange(dim) + 0.5
        xx, yy = np.meshgrid(x, y)
        xx = xx.reshape(-1)
        yy = yy.reshape(-1)
        centers = np.array([xx, yy]).T
    if topology == 'hexagonal':
        yy, xx= np.meshgrid(np.arange(dim), np.arange(dim))
        shift = np.zeros(dim)
        shift[::2]=-0.5
        xx = xx + shift
        yy = yy * (np.sqrt(3) / 2)
        centers = np.array([xx.reshape(-1), yy.reshape(-1)]).T
    return centers

def get_cell_boundary(center, topology='rectangular'):
    if topology == 'rectangular':
        points = [np.array([[center[0]-0.5, center[1]-0.5], [center[0]-0.5, center[1]+0.5]]),
                 np.array([[center[0]-0.5, center[1]+0.5], [center[0]+0.5, center[1]+0.5]]),
                np.array([[center[0]+0.5, center[1]-0.5], [center[0]+0.5, center[1]+0.5]]),
                 np.array([[center[0]-0.5, center[1]-0.5], [center[0]+0.5, center[1]-0.5]])]
        return points
    elif topology == 'hexagonal':
        dx = 0.5
        dy = np.sqrt(3)/6
        points = [np.array([[center[0]-dx, center[1]+dy], [center[0], center[1]+2*dy]]),
                  np.array([[center[0], center[1]+2*dy], [center[0]+dx, center[1]+dy]]),
                 np.array([[center[0]+dx, center[1]-dy], [center[0]+dx, center[1]+dy]]),
                 np.array([[center[0], center[1]-2*dy], [center[0]+dx, center[1]-dy]]),
                 np.array([[center[0]-dx, center[1]-dy], [center[0], center[1]-2*dy]]),
                 np.array([[center[0]-dx, center[1]-dy], [center[0]-dx, center[1]+dy]])]
    return points

def get_manycells_boundary(centers, topology='rectangular'):
    points=[]
    for center in centers:
        points+=get_cell_boundary(center, topology)
    points_unique, counts = np.unique(np.round(np.array(points),4), axis=0, return_counts=True)
    return points_unique[counts==1]


Now we calculate and plot the SOM grid color-coded by the average true redshift of the target sample (left panel) and the reference sample (right panel) to show the reference sample is indeed representing the redshift of the target galaxies

In [ ]:
mean_photZ_ref = np.zeros_like(SOM.umatrix).reshape(-1)  # mean photometric redshift of reference data in each cell/cluster
mean_specZ_ref = np.zeros_like(SOM.umatrix).reshape(-1)  # mean spectroscopic redshift of reference data in each cell/cluster

mean_photZ_target = np.zeros_like(SOM.umatrix).reshape(-1)   # mean photometric redshift of target data in each cell/cluster
mean_specZ_target = np.zeros_like(SOM.umatrix).reshape(-1)   # mean spectroscopic redshift of target data in each cell/cluster

for i in range(n_clusters):
    
    mean_photZ_ref[som_cluster_inds==i] = np.median(bpz_spec[ref_som_clusterind==i])
    mean_specZ_ref[som_cluster_inds==i] = np.median(ref_data.data['photometry']['redshift'][ref_som_clusterind==i])
    
    mean_photZ_target[som_cluster_inds==i] = np.median(bpz_phot[target_som_clusterind==i])
    mean_specZ_target[som_cluster_inds==i] = np.median(target_data.data['photometry']['redshift'][target_som_clusterind==i])

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
plot_som(ax[0], mean_specZ_target.reshape(dim, dim), grid_type=grid_type, colormap=cm.coolwarm, cbar_name='mean true redshift of the target sample', vmin=bin_low, vmax=bin_high)
ax[0].set_title('Target sample')

plot_som(ax[1], mean_specZ_ref.reshape(dim, dim), grid_type=grid_type, colormap=cm.coolwarm, cbar_name='mean true redshift of the reference sample', vmin=bin_low, vmax=bin_high)

ax[1].set_title('Reference sample')
plot_cluster_boundaries(ax[0], SOM, n_clusters)

# Now let's do the quality control.

Quality control means selecting a subset of the clusters/SOM cells where the target galaxies are well represented by the reference sample.
We evaluate the "goodness-of-reference" by the two criteria given by https://arxiv.org/pdf/2007.15635 

Quality cut 1 (QC1):

$\frac{\left|\left\langle z_{\text {spec }}\right\rangle-\left\langle Z_{\mathrm{B}}\right\rangle\right|}{\operatorname{nMAD}\left(\left\langle z_{\text {spec }}\right\rangle-\left\langle Z_{\mathrm{B}}\right\rangle\right)}>5$

This QC will remove outliers in the distribution of photo-$z$.

Quality cut 2 (QC2):

$\left|\left\langle Z_B\right\rangle_{\text {spec }}-\left\langle Z_B\right\rangle_{\text {phot }}\right|>0.02$

This QC will remove clusters in which the target and reference sample have very different photo-$z$, which means the reference galaxies are not representative.

We also try to combine these two QCs.

In [ ]:
from scipy.stats import median_abs_deviation

zmean_diff_cluster_qc1 = np.zeros(n_clusters)
zmean_diff_cluster_qccombined = np.zeros(n_clusters)

for i in range(n_clusters):   
    
    zmean_diff_cluster_qc1[i] = np.fabs(np.median(ref_data.data['photometry']['redshift'][ref_som_clusterind==i]) - np.median(bpz_phot[target_som_clusterind==i]))
        
    zmean_diff_cluster_qccombined[i] = np.fabs(np.median(bpz_phot[target_som_clusterind==i])-np.median(bpz_spec[ref_som_clusterind==i]))
    

cluster_ind_good = np.where(~np.isnan(zmean_diff_cluster_qccombined))
cluster_ind_good_qc1 = np.where(zmean_diff_cluster_qc1<max(median_abs_deviation((mean_specZ_ref-mean_photZ_ref)[~np.isnan(mean_specZ_ref-mean_photZ_ref)])*5, 0))
cluster_ind_good_qc2 = np.where(zmean_diff_cluster_qccombined<0.02)

cluster_ind_good_qccombined = np.intersect1d(cluster_ind_good_qc2, cluster_ind_good_qc1)

Here is the cluster occupation distribution:

And here is the mean redshift per cluster:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,5))

plot_som(ax[0], np.fabs(mean_specZ_ref-mean_photZ_target).reshape(dim,dim)/max(median_abs_deviation((mean_specZ_ref-mean_photZ_ref)[~np.isnan(mean_specZ_ref-mean_photZ_ref)])*5, 0), grid_type=grid_type, colormap=cm.coolwarm, cbar_name=r'$\frac{\left|\left\langle z_{\text {spec }}\right\rangle-\left\langle Z_{\mathrm{B}}\right\rangle\right|}{\operatorname{nMAD}\left(\left\langle z_{\text {spec }}\right\rangle-\left\langle Z_{\mathrm{B}}\right\rangle\right)\times 5}$', vmin=0, vmax=1)

plot_cluster_boundaries(ax[0], SOM, n_clusters, cluster_inds=cluster_ind_good_qc1[0])

zmean_diff = np.fabs(mean_specZ_target - mean_specZ_ref).reshape(dim, dim)

plot_som(ax[1], (zmean_diff)/0.02, grid_type=grid_type, colormap=cm.coolwarm, cbar_name=r'$\left|\left\langle Z_B\right\rangle_{\text {spec }}-\left\langle Z_B\right\rangle_{\text {phot }}\right|/0.02$', vmin=0, vmax=1)

plot_cluster_boundaries(ax[1], SOM, n_clusters, cluster_inds=cluster_ind_good_qc2[0])


Now that we have illustrated what exactly we have constructed, let's use the SOM to predict the redshift distribution for a set of photometric objects.  
We first summarize the case withouth QC, then the three QCs (QC1, QC2, QC1+QC2)

Note that we have removed the 'photometry' group, we will specify the `phot_groupname` as "" in the parameters below.<br>
As before, let us specify our initialization params for the SOMocluSummarizer stage, including:<br>
`model`: name of the pickled model that we created, in this case "output_SOM_model.pkl"<br>
`hdf5_groupname` (str): hdf5 group for our photometric data (in our case "")<br>
`objid_name` (str): string specifying the name of the ID column, if present photom data, will be written out to cellid_output file<br>
`spec_groupname` (str): hdf5 group for the spectroscopic data<br>
`nzbins` (int): number of bins to use in our histogram ensemble<br>
`n_clusters` (int): number of hierarchical clusters<br>
`nsamples` (int): number of bootstrap samples to generate<br>
`output` (str): name of the output qp file with N samples<br>
`single_NZ` (str): name of the qp file with fiducial distribution<br>
`uncovered_cell_file` (str): name of hdf5 file containing a list of all of the cells with phot data but no spec-z objects: photometric objects in these cells will *not* be accounted for in the final N(z), and should really be removed from the sample before running the summarizer.  Note that we return a single integer that is constructed from the pairs of SOM cell indices via `np.ravel_multi_index`(indices).<br>

Now let's initialize and run the summarizer.  One feature of the SOM: if any SOM cells contain photometric data but do not contain any redshifts values in the spectroscopic set, then no reasonable redshift estimate for those objects is defined, and they are skipped.  The method currently prints the indices of uncovered cells, we may modify the algorithm to actually output the uncovered galaxies in a separate file in the future.

Let's open the fiducial N(z) file, plot it, and see how it looks, and compare it to the true tomographic bin file:

In [ ]:
boot_ens = qp.read('SOM_ensemble.hdf5')

target_nz_hist, zbin = get_cont_hist(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good)], np.linspace(0,3,101))

fid_ens = qp.read('fiducial_SOMoclu_NZ.hdf5')
som_nz_hist = np.squeeze(fid_ens.pdf(zbin))

full_ens = qp.read("SOM_ensemble.hdf5")
full_means = full_ens.mean().flatten()
full_stds = full_ens.std().flatten()
true_full_mean = np.mean(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good)])
true_full_std = np.std(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good)])
# mean and width of bootstraps
full_mu = np.mean(full_means)
full_sig = np.std(full_means)
full_norm = norm(loc=full_mu, scale=full_sig)
grid = np.linspace(0, .7, 301)
full_uncert = full_norm.pdf(grid)*2.51*full_sig

print('===========This is the fiducial case==================')

print("The mean redshift of the SOM ensemble is: "+str(round(np.mean(full_means),4)) + '+-' + str(round(np.std(full_means),4)))
print("The mean redshift of the real data is: "+str(round(true_full_mean,4)))
print("The bias of mean redshift is:"+str(round(np.mean(full_means)-true_full_mean,4)) + '+-' + str(round(np.std(full_means),4)))

In [ ]:
som_model_qc1=DS.read_file('som_model_qc1',  ModelHandle, "output_SOMoclu_model.pkl",)

summ_dict_qc1 = dict(model=som_model_qc1, hdf5_groupname='photometry',
                 spec_groupname='photometry', nzbins=101, nsamples=25,
                 output='SOM_ensemble_qc1.hdf5', single_NZ='fiducial_SOMoclu_NZ_qc1.hdf5',
                 n_clusters=n_clusters,
                 uncovered_cell_file='all_uncovered_cells_qc1.hdf5',
                 objid_name='id',
                 cellid_output='output_cellIDs_qc1.hdf5', useful_clusters=cluster_ind_good_qc1[0])

som_summarizer_qc1 = SOMocluSummarizer.make_stage(name='SOMoclu_summarizer_qc1', **summ_dict_qc1)
som_summarizer_qc1.summarize(target_data, ref_data)

fid_ens_qc1 = qp.read("fiducial_SOMoclu_NZ_qc1.hdf5")
boot_ens_qc1 = qp.read('SOM_ensemble_qc1.hdf5')
target_nz_hist_qc1, zbin_qc1 = get_cont_hist(target_data.data['photometry']['redshift'], np.linspace(0,3,101))
som_nz_hist_qc1 = np.squeeze(fid_ens_qc1.pdf(zbin))

target_nz_hist_qc1_true, zbin = get_cont_hist(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc1)], np.linspace(0,3,101))

full_ens_qc1 = qp.read("SOM_ensemble_qc1.hdf5")
full_means_qc1 = full_ens_qc1.mean().flatten()
full_stds_qc1 = full_ens_qc1.std().flatten()
true_full_mean_qc1 = np.mean(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc1)])
true_full_std_qc1 = np.std(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc1)])
# mean and width of bootstraps
full_mu_qc1 = np.mean(full_means_qc1)
full_sig_qc1 = np.std(full_means_qc1)
full_norm_qc1 = norm(loc=full_mu_qc1, scale=full_sig_qc1)
grid = np.linspace(0, .7, 301)
full_uncert_qc1 = full_norm_qc1.pdf(grid)*2.51*full_sig_qc1

print('\n\n===========This is the QC1 case==================')

print("The mean redshift of the SOM ensemble is: "+str(round(np.mean(full_means_qc1),4)) + '+-' + str(round(np.std(full_means_qc1),4)))
print("The mean redshift of the real data is: "+str(round(true_full_mean_qc1,4)))
print("The bias of mean redshift is:"+str(round(np.mean(full_means_qc1)-true_full_mean_qc1,4)) + '+-' + str(round(np.std(full_means_qc1),4)))

In [ ]:
from scipy.stats import norm

som_model_qc2=DS.read_file('som_model_qc2',  ModelHandle, "output_SOMoclu_model.pkl",)

summ_dict_qc2 = dict(model=som_model_qc2, hdf5_groupname='photometry',
                 spec_groupname='photometry', nzbins=101, nsamples=25,
                 output='SOM_ensemble_qc2.hdf5', single_NZ='fiducial_SOMoclu_NZ_qc2.hdf5',
                 n_clusters=n_clusters,
                 uncovered_cell_file='all_uncovered_cells_qc2.hdf5',
                 objid_name='id',
                 cellid_output='output_cellIDs_qc2.hdf5', useful_clusters=cluster_ind_good_qc2[0])

som_summarizer_qc2 = SOMocluSummarizer.make_stage(name='SOMoclu_summarizer_qc2', **summ_dict_qc2)
som_summarizer_qc2.summarize(target_data, ref_data)

fid_ens_qc2 = qp.read("fiducial_SOMoclu_NZ_qc2.hdf5")
boot_ens_qc2 = qp.read('SOM_ensemble_qc2.hdf5')
target_nz_hist_qc2, zbin_qc2 = get_cont_hist(target_data.data['photometry']['redshift'], np.linspace(0,3,101))
som_nz_hist_qc2 = np.squeeze(fid_ens_qc2.pdf(zbin))
target_nz_hist_qc2_true, zbin = get_cont_hist(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc2)], np.linspace(0,3,101))

full_ens_qc2 = qp.read("SOM_ensemble_qc2.hdf5")
full_means_qc2 = full_ens_qc2.mean().flatten()
full_stds_qc2 = full_ens_qc2.std().flatten()
true_full_mean_qc2 = np.mean(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc2)])
true_full_std_qc2 = np.std(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qc2)])
# mean and width of bootstraps
full_mu_qc2 = np.mean(full_means_qc2)
full_sig_qc2 = np.std(full_means_qc2)
full_norm_qc2 = norm(loc=full_mu_qc2, scale=full_sig_qc2)
grid = np.linspace(0, .7, 301)
full_uncert_qc2 = full_norm_qc2.pdf(grid)*2.51*full_sig_qc2

print("\n\n===========This is the QC2 case==================")

print("The mean redshift of the SOM ensemble is: "+str(round(np.mean(full_means_qc2),4)) + '+-' + str(round(np.std(full_means_qc2),4)))
print("The mean redshift of the real data is: "+str(round(true_full_mean_qc2,4)))
print("The bias of mean redshift is:"+str(round(np.mean(full_means_qc2)-true_full_mean_qc2,4)) + '+-' + str(round(np.std(full_means_qc2),4)))

In [ ]:
som_model_qccombined=DS.read_file('som_model_qccombined',  ModelHandle, "output_SOMoclu_model.pkl",)

summ_dict_qccombined = dict(model=som_model_qccombined, hdf5_groupname='photometry',
                 spec_groupname='photometry', nzbins=101, nsamples=25,
                 output='SOM_ensemble_qccombined.hdf5', single_NZ='fiducial_SOMoclu_NZ_qccombined.hdf5',
                 n_clusters=n_clusters,
                 uncovered_cell_file='all_uncovered_cells_qccombined.hdf5',
                 objid_name='id',
                 cellid_output='output_cellIDs_qccombined.hdf5', useful_clusters=cluster_ind_good_qccombined)

som_summarizer_qccombined = SOMocluSummarizer.make_stage(name='SOMoclu_summarizer_qccombined', **summ_dict_qccombined)
som_summarizer_qccombined.summarize(target_data, ref_data)

fid_ens_qccombined = qp.read("fiducial_SOMoclu_NZ_qccombined.hdf5")
boot_ens_qccombined = qp.read('SOM_ensemble_qccombined.hdf5')
target_nz_hist_qccombined, zbin_qccombined = get_cont_hist(target_data.data['photometry']['redshift'], np.linspace(0,3,101))
som_nz_hist_qccombined = np.squeeze(fid_ens_qccombined.pdf(zbin))
target_nz_hist_qccombined_true, zbin = get_cont_hist(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qccombined)], np.linspace(0,3,101))

full_ens_qccombined = qp.read("SOM_ensemble_qccombined.hdf5")
full_means_qccombined = full_ens_qccombined.mean().flatten()
full_stds_qccombined = full_ens_qccombined.std().flatten()
true_full_mean_qccombined = np.mean(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qccombined)])
true_full_std_qccombined = np.std(target_data.data['photometry']['redshift'][np.in1d(target_som_clusterind, cluster_ind_good_qccombined)])
# mean and width of bootstraps
full_mu_qccombined = np.mean(full_means_qccombined)
full_sig_qccombined = np.std(full_means_qccombined)
full_norm_qccombined = norm(loc=full_mu_qccombined, scale=full_sig_qccombined)
grid = np.linspace(0, .7, 301)
full_uncert_qccombined = full_norm_qccombined.pdf(grid)*2.51*full_sig_qccombined

print("\n\n===========This is the QC1+QC2 case==================")

print("The mean redshift of the SOM ensemble is: "+str(round(np.mean(full_means_qccombined),4)) + '+-' + str(round(np.std(full_means_qccombined),4)))
print("The mean redshift of the real data is: "+str(round(true_full_mean_qccombined,4)))
print("The bias of mean redshift is:"+str(round(np.mean(full_means_qccombined)-true_full_mean_qccombined,4)) + '+-' + str(round(np.std(full_means_qccombined),4)))

Now we plot the true/calibrated redshift distributions of the four cases

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(24,12))
ax = ax.flatten()
ax[0].set_xlabel("redshift", fontsize=15)
ax[0].set_ylabel("N(z)", fontsize=15)
ax[0].plot(zbin, target_nz_hist, label='True N(z)')
ax[0].plot(zbin, som_nz_hist,  color='C1',label='SOM N(z)')

for i in range(boot_ens.npdf):
    #ax = plt.subplot(2,3,i+1)
    pdf = np.squeeze(boot_ens[i].pdf(zbin))
    if i == 0:        
        ax[0].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)
    else:
        ax[0].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)

ax[1].set_title(f'{round(som_summarizer_qc1.neff_p_to_neff / som_summarizer.neff_p_to_neff*100, 2)}% galaxies kept')
ax[1].plot(zbin_qc1, target_nz_hist_qc1_true, color='C0',label='True N(z), QC1 selected')
ax[1].plot(zbin_qc1, som_nz_hist_qc1, color='C1',label='SOM N(z), QC1 selected')
ax[1].set_xlabel("redshift", fontsize=15)
ax[1].set_ylabel("N(z)", fontsize=15)
for i in range(boot_ens.npdf):
    #ax = plt.subplot(2,3,i+1)
    pdf = np.squeeze(boot_ens_qc1[i].pdf(zbin))
    if i == 0:        
        ax[1].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)
    else:
        ax[1].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)

ax[2].set_title(f'{round(som_summarizer_qc2.neff_p_to_neff / som_summarizer.neff_p_to_neff*100, 2)}% galaxies kept')
ax[2].plot(zbin_qc2, target_nz_hist_qc2_true, color='C0',label='True N(z), QC2 selected')
ax[2].plot(zbin_qc2, som_nz_hist_qc2, color='C1',label='SOM N(z), QC2 selected')
ax[2].set_xlabel("redshift", fontsize=15)
ax[2].set_ylabel("N(z)", fontsize=15)
for i in range(boot_ens.npdf):
    #ax = plt.subplot(2,3,i+1)
    pdf = np.squeeze(boot_ens_qc2[i].pdf(zbin))
    if i == 0:        
        ax[2].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)
    else:
        ax[2].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)

ax[3].set_title(f'{round(som_summarizer_qccombined.neff_p_to_neff / som_summarizer.neff_p_to_neff*100, 2)}% galaxies kept')
ax[3].plot(zbin_qccombined, target_nz_hist_qccombined_true, color='C0',label='True N(z), QC1+2 selected')
ax[3].plot(zbin_qccombined, som_nz_hist_qccombined, color='C1',label='SOM N(z), QC1+2 selected')
ax[3].set_xlabel("redshift", fontsize=15)
ax[3].set_ylabel("N(z)", fontsize=15)
for i in range(boot_ens.npdf):
    #ax = plt.subplot(2,3,i+1)
    pdf = np.squeeze(boot_ens_qccombined[i].pdf(zbin))
    if i == 0:        
        ax[3].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)
    else:
        ax[3].plot(zbin, pdf, color='C1',zorder=0, alpha=0.2)

ax[0].legend(fontsize=15)
ax[1].legend(fontsize=15)
ax[2].legend(fontsize=15)
ax[3].legend(fontsize=15)
ax[0].set_xlim(0,1.5)
ax[1].set_xlim(0,1.5)
ax[2].set_xlim(0,1.5)
ax[3].set_xlim(0,1.5)

The following box plot summarizes the bias in the redshift estimation for the four cases

In [ ]:
data = [full_means_qccombined-true_full_mean_qccombined, full_means_qc2-true_full_mean_qc2, full_means_qc1-true_full_mean_qc1, full_means-true_full_mean]

#plt.boxplot(data)
fig, ax = plt.subplots(1,1)
ax.boxplot(data, vert = 0)
ax.axvline(0,1,0, color='C0')
ax.set_xlim(-0.02,0.02)
ax.set_xlabel(r'$\Delta\left\langle z \right\rangle$')
ax.set_yticklabels(['SOM N(z), QC1+QC2', 'SOM N(z), QC2','SOM N(z), QC1', 'SOM N(z)'])

In [ ]:
import sys
#sys.path.append('/net/home/fohlen14/yanza21/research/src/RAIL_branches/RAIL/src/rail/estimation')

#from algos.somocluSOM import SOMocluSummarizer

n_clusterss = np.linspace(50, 1500, 10, dtype=int)

true_full_mean = np.mean(target_data.data['photometry']['redshift'])
true_full_std = np.std(target_data.data['photometry']['redshift'])
mu_diff = np.zeros(n_clusterss.size)
means_diff = np.zeros((n_clusterss.size, 50))

std_diff_mean = np.zeros(n_clusterss.size)
neff_p_to_neff = np.zeros(n_clusterss.size)
#neff_p_to_neff = np.zeros((n_clusterss.size, 25))
std_diff = np.zeros((n_clusterss.size, 50))
for i, n_clusters_ in enumerate(n_clusterss):
    summ_dict = dict(model="output_SOMoclu_model.pkl", hdf5_groupname='photometry',
                 spec_groupname='photometry', nzbins=101, nsamples=50,
                 output='SOM_ensemble.hdf5', single_NZ='fiducial_SOMoclu_NZ.hdf5',
                 n_clusters=n_clusters_,
                 uncovered_cluster_file ='all_uncovered_cells.hdf5',
                 objid_name='id',
                 cellid_output='output_cellIDs.hdf5')
    som_summarizer = SOMocluSummarizer.make_stage(name='SOMoclu_summarizer', aliases=dict(model="model_somoclu"), **summ_dict)    
    som_summarizer.summarize(target_data, ref_data)
    
    full_ens = qp.read("SOM_ensemble.hdf5")
    full_means = full_ens.mean().flatten()
    full_stds = full_ens.std().flatten()
    
    # mean and width of bootstraps
    mu_diff[i] = np.mean(full_means) - true_full_mean
    means_diff[i] = full_means - true_full_mean
    
    std_diff_mean[i] = np.mean(full_stds) - true_full_std
    std_diff[i] = full_stds - true_full_std
    neff_p_to_neff[i] = som_summarizer.neff_p_to_neff
    #neff_p_to_neff_mean[i] = np.mean(som_summarizer.neff_p_to_neff)
    full_sig = np.std(full_means)
    


In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(20,5))

'''for i in range(25):
    axes[0].plot(n_clusterss, means_diff.T[i], lw=0.2, color='C1')'''
axes[0].plot(n_clusterss, mu_diff, lw=1, color='k')
axes[0].axhline(0,1,0)
axes[0].set_xlabel('Number of clusters')
axes[0].set_ylabel(r'$\left\langle z \right\rangle - \left\langle z \right\rangle_{\mathrm{true}}$')

'''for i in range(25):
    axes[1].plot(n_clusterss, std_diff.T[i], lw=0.2, color='C1')'''
axes[1].plot(n_clusterss, std_diff_mean, lw=1, color='k')
axes[1].axhline(0,1,0)

axes[1].set_xlabel('Number of clusters')
axes[1].set_ylabel(r'$\mathrm{std}(z) - \mathrm{std}(z)_{\mathrm{true}}$')


'''for i in range(25):
    axes[2].plot(n_clusterss, neff_p_to_neff.T[i]*100, lw=0.2, color='C1')'''
axes[2].plot(n_clusterss, neff_p_to_neff*100, lw=1, color='k')

axes[2].set_xlabel('Number of clusters')
axes[2].set_ylabel(r'$n_{\mathrm{eff}}\'/n_{\mathrm{eff}}$(%)')